# Data preparation

#### Import packages 

In [6]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
plt.style.use('seaborn-poster')

#### Load data sets 

In [7]:
order_book = pd.read_csv("clean_order_book.csv")
msg_book = pd.read_csv("clean_msg_book.csv")

#### Add midprice variations to have data labels and use supervised algorithms

In [50]:
dataset = order_book
dataset['MidPrice'] = dataset.loc[:, ['ASKp1', 'BIDp1']].mean(1)
dataset['MidPriceT+1'] = dataset['MidPrice'].shift(-1)
dataset['MidPriceVariation'] = dataset['MidPriceT+1'] - dataset['MidPrice']
dataset = dataset.dropna()
print(dataset.shape)

(269746, 43)


####  Check the ratio of deep events (orders in levels different than ASK1 and BID1)

In [46]:
np.count_nonzero(dataset['MidPriceVariation'] == 0) / dataset['MidPriceVariation'].shape[0]

0.8978446390307919

#### Drop all those deep events (noise for mid price up/down classification)
We have 10% of the data remaining

In [52]:
dataset = dataset[dataset['MidPriceVariation'] != 0]
dataset['binaryOutput'] = (dataset['MidPriceVariation'] > 0).astype(int)
print(dataset.shape)

(27556, 44)


/home/leo/.envs/global/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### Save prepared data set 

In [81]:
dataset.to_csv("prepared_dataset.csv", index=False)

#### Build input matrix, output label 

In [80]:
import data_formatter

X, y = data_formatter.prepare_data_set(dataset, 10)
print(X.shape)

(27556, 400)
